In [1]:
using Pkg
Pkg.activate("/home/zhenan/projects/def-mpf/zhenan/julia/dev/AtomicOpt")

  Activating environment at `~/projects/def-mpf/zhenan/julia/dev/AtomicOpt/Project.toml`


In [2]:
using Revise
using AtomicOpt
using LinearAlgebra
using SparseArrays
using Printf
using Arpack

┌ Info: Precompiling AtomicOpt [03e163f5-eebc-44fa-8de0-41458aa85bdf]
└ @ Base loading.jl:1342


## Load data

In [19]:
function load_data(data::String)
    if data == "temperature"
        file = open("./temp2020.csv", "r")
        I = Vector{Int64}(); J = Vector{Int64}(); V = Vector{Float64}()
        numRows=0; RowDict = Dict{String, Int64}();
        numCols=0; ColDict = Dict{String, Int64}();
        while !eof(file)
            line = readline(file)
            info = split(line, ",")
            # get value
            v = parse(Float64, info[3])
            if v!= 0
                push!(V, v)
                # get row index
                if !haskey(RowDict, info[1])
                    numRows += 1
                    RowDict[info[1]] = numRows
                end
                i = RowDict[info[1]]
                push!(I, i)
                # get column index
                if !haskey(ColDict, info[2])
                    numCols += 1
                    ColDict[info[2]] = numCols
                end
                j = ColDict[info[2]]
                push!(J, j)
            end    
        end
        close(file)
        X = sparse(I,J,V,numRows,numCols)
        X ./= maximum(X)
        # low rank approximation
        m, n = size(X)
        r = 5
        U, S, V = svds(X, nsv=r)[1]
        Xopt = U*Diagonal(S)*V'
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement
        b = Vector{Float64}()
        bopt = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
            push!(bopt, Xopt[i,j])
        end
        # optimal parameters
        τ = sum(S)
        z = b - bopt
        α = norm(z)^2/2
        Z = sparse(I, J, z)
        λ = dot(Z, Xopt)/τ
        return m, n, r, Xopt, mask, b, α, τ, λ
    elseif data == "movie"
        file = open("./ml-100k.txt", "r")
        I = Vector{Int64}(); J = Vector{Int64}(); V = Vector{Float64}()
        while !eof(file)
            line = readline(file)
            info = split(line)
            i = parse(Int64, info[1])
            j = parse(Int64, info[2])
            v = parse(Float64, info[3])
            push!(I, i); push!(J, j); push!(V, v)    
        end
        close(file)
        X = sparse(I,J,V,maximum(I),maximum(J))
        X ./= maximum(X)
        # low rank approximation
        m, n = size(X)
        r = 5
        U, S, V = svds(X, nsv=r)[1]
        Xopt = U*Diagonal(S)*V'
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement
        b = Vector{Float64}()
        bopt = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
            push!(bopt, Xopt[i,j])
        end
        # optimal parameters
        τ = sum(S)
        z = b - bopt
        α = norm(z)^2/2
        Z = sparse(I, J, z)
        λ = dot(Z, Xopt)/τ
        return m, n, r, Xopt, mask, b, α, τ, λ
    elseif data == "random"
        m, n, r = 100, 100, 3
        X = rand(m, n)
        U, S, V = svd(X)
        X = U[:,1:r] * Diagonal( S[1:r] ) * V[:,1:r]'
        # mask 
        p = 0.5
        mask = sprand(Bool, m, n, p)
        mask = convert(SparseMatrixCSC{Float64, Int64}, mask)
        I,J,V = findnz(mask)
        # measurement 
        b = Vector{Float64}()
        l = length(I)
        for t in 1:l
            i, j = I[t], J[t]
            push!(b, X[i,j])
        end
        η = rand(l)
        ϵ = 1; η .*= (ϵ/norm(η))
        η = zeros(l)
        b .+= η
        # optimal parameters
        τ = sum(S[1:r])
        α = norm(η)^2/2
        Z = sparse(I, J, η)
        λ = dot(Z, X)/τ
        return m, n, r, X, mask, b, α, τ, λ
    else
        println("invalid data")
        return
    end
end

load_data (generic function with 1 method)

In [20]:
m, n, r, Xopt, mask, b, αopt, τopt, λopt = load_data("random");

In [21]:
τopt

60.97867859919652

In [22]:
αopt

0.0

## Solve matrix completion problem

In [24]:
Mop = MaskOP(mask)
A = NucBall(m, n, r)
sol = level_set(Mop, b, A; 
    α=αopt, tol=1e-5, maxIts=100*length(b), callback=(dcg->dual_obj_gap(dcg, τopt, λopt, αopt)), pr=true);


  -------------------------------------------------------------------------
  Polar Level Set Method
  -------------------------------------------------------------------------
  number of variables      10000         number of constraints    4984
  feasibility tolerance  3.67e-04         α                    0.00e+00
  max iterations          498400 
  -------------------------------------------------------------------------
  Major      Minor        u-α        ℓ-α        gap          τ         infeas-α  Subproblem
callback(dcg) = 8.714087574848612
      1          2   1.66e+02   1.62e+02   3.20e+00   2.54e+01       6.38e+02   suboptimal
callback(dcg) = 5.929776413852402
      2          1   4.76e+01   4.35e+01   4.11e+00   3.86e+01       6.38e+02   suboptimal
callback(dcg) = 1.9598985031859115
      3          3   1.82e+01   1.36e+01   4.65e+00   4.62e+01       3.61e+00   suboptimal
callback(dcg) = 5.094707264127891
      4          3   7.83e+00   3.69e+00   4.14e+00   5.14e+01     

callback(dcg) = 1.0965930201807623
     63        107   7.35e-02   2.09e-03   7.14e-02   6.03e+01       7.99e-03   suboptimal
callback(dcg) = 1.5484416903565261
     64        161   6.97e-02   1.75e-03   6.79e-02   6.03e+01       7.99e-03   suboptimal
callback(dcg) = 1.30170680228008
     65         29   6.77e-02   4.14e-04   6.73e-02   6.03e+01       7.99e-03   suboptimal
callback(dcg) = 0.8536143320053071
     66         65   6.66e-02   5.28e-04   6.61e-02   6.03e+01       7.99e-03   suboptimal
callback(dcg) = 1.5462694138456783
     67         19   6.59e-02   8.40e-04   6.50e-02   6.03e+01       7.99e-03   suboptimal
callback(dcg) = 1.1648315911666955
     68         49   6.46e-02   1.42e-03   6.32e-02   6.03e+01       7.99e-03   suboptimal
callback(dcg) = 1.5078895902644618
     69         29   6.30e-02   1.40e-03   6.16e-02   6.04e+01       7.99e-03   suboptimal
callback(dcg) = 1.2654505349588803
     70         63   6.11e-02   8.57e-04   6.02e-02   6.04e+01       7.99e-03   subop

## Report relative difference

In [25]:
x = constructPrimal(sol)
X = reshape(x, m, n)
@printf "The relative difference between Xopt and X: %e \n" norm(Xopt - X)/norm(Xopt)

The relative difference between Xopt and X: 7.855087e-04 
